[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/CCS-ZCU/pribehy-dat/blob/master/scripts/nlp.ipynb)

# NLP: Zpracování přirozeného jazyka

**autor**: *Vojtěch Kaše* (kase@ff.zcu.cz)

[![](https://ccs.zcu.cz/wp-content/uploads/2021/10/cropped-ccs-logo_black_space_240x240.png)](https://ccs.zcu.cz)

## Úvod a cíle kapitoly

Cílem tohoto cvičení je provést základní kvantitativní textovou analýzu některého digitalizovaného dokumentu z **Archivu Jana Patočky** ([AJP](https://archiv.janpatocka.cz/items/browse?tags=fulltext)). Omezíme se však pouze na dokumenty, u kterých je dostupný digitální přepis (tzv. fulltext). Tj. URL adresa, kterou hledáme, je adresa jakéhokoliv námi vybraného dokumentu z daného archivu pro který je dostupný přepis. Pro zvládnutí cvičení není potřeba nic více než na několika místech ručně nahradit jeden řetězec znaků za jiný pomocí Ctrl+C a Ctrl+V a ve správném pořadí spustit jednotlivé buňky pomocí ikony "play" či klávesové zkratky Shift+Enter.

Doklad o provedení cvičení bude nahrání Vámi upravené kopie jupyter notebooku ve formátu PDF do odevzdávárny v moodlu (viz níže).

Následující buňka je první klíčovou buňkou pro naše cvičení a  je potřeba ji spustit předtím, než buňky, které následují, neboť ty staví na funkcích, které v této buňce do aktuálního jupyter notebooku importujeme.

## Cvičení

In [ ]:
# naše první buňka slouží k tomu, abychom do aktuálního GoogleColab skriptu
# importovali určité dodatečné balíčky (či knihovny) s funkcemi,
# které nejsou v "základní výbavě" Pythonu. Některé knihovny budeme muset dokonce nainstalovat

import requests
from urllib.request import urlopen # pro práci w webovými adresami
from urllib.parse import quote
from bs4 import BeautifulSoup # pro práci s webovými stránkami ve formátu html
import json
import re # pro práci s regulérními výrazy
import pandas as pd # pro práci s tabulkami ve formátu dataframe
import nltk # modul pro práci s textovými daty
from nltk import tokenize
nltk.download('punkt')
import matplotlib.pyplot as plt # modul pro vytváření grafů
import numpy as np # modul pro pokročilejší matematické operac

Váš hlavní úkol je spojen s buňkou níže. V ní je potřeba nahradit obsah proměnné "url", tj. **vyměnit webovou adresu jednoho dokumentu z AJP za adresu jiného dokumentu z téhož archivu**.  Pozor, že webová adresa musí být uvnitř uvozovek. Aby se změna projevila, je třeba buňku nakonec spustit.

In [ ]:
url = "https://archiv.janpatocka.cz/items/show/308"
web_text = urlopen(url).read().decode("utf-8")
soup = BeautifulSoup(web_text, "html.parser")

In [ ]:
text_title = soup.find("div", id="item_title").get_text()
text_title

In [ ]:
[div for div in soup.find_all("div", class_="col span_7_of_9")][3].get_text()

In [ ]:
text_dokumentu = soup.find("div", id="trans_full").get_text()
text_dokumentu = " ".join(text_dokumentu.split())
print(text_dokumentu)

In [ ]:
# dokument jako list slov získáme pomocí funkce "split()"
# uložíme si ho takto do nové proměnné "pdf_string_list"
string_list = text_dokumentu.split()
# tuto proměnnou si vypíšeme:
string_list[:20]

In [ ]:
### pomocí funkce "len()" spočítáme délku tohoto listu slov:
len(string_list)

In [ ]:
data = []
for n in range(0,3000):
  try:
    textdata = {}
    id = str(n)
    url = "https://archiv.janpatocka.cz/items/show/" + id
    web_text = urlopen(url).read().decode("utf-8")
    soup = BeautifulSoup(web_text, "html.parser")
    text_title = soup.find("div", id="item_title").get_text()
    date = [div for div in soup.find_all("div", class_="col span_7_of_9")][3].get_text()
    text_dokumentu = soup.find("div", id="trans_full").get_text()
    text_dokumentu = " ".join(text_dokumentu.split())
    print(id)
    textdata["id"] = id
    textdata["title"] = text_title
    textdata["date"] = date
    textdata["rawtext"] = text_dokumentu
    data.append(textdata)
  except:
    pass




In [ ]:
data_df = pd.DataFrame(data)
data_df.head(5)

In [ ]:
data_df.to_csv("my_table.csv")

### Lematizace a postagging

S textem článku, tak jak se nyní nachází v proměnné "text_clanku", bychom se ale při kvantitativní textové analýze stále příliš daleko nedostali. Čeština je totiž morfologicky velice bohatý jazyk. Chceme-li např. spočítat kolikrát se v textu objevuje sloveso "mít", s textem v aktuální podobě se příliš daleko nedstaneme. Zde potřebujeme na naše textová data aplikovat dvě další procedury:


1.   lemmatizace, tj. převedení slov z textu do jejich základních tvarů (slovesa do infinitivu, podstatná jména do 1.pádu singuláru apod.)
2.   POS-tagging ("part-of-speech tags"),  tj. určení slovních druhů a mluvnických kategorií

Aplikace těchto procedur nám umožní získat data z hlediska kvantitativní textové analýzy výrazně zajímavější.

V případě češtiny se můžeme v tomto případě opřít o webové nástroje z dílny Ústavu formální a aplikované lingivistiky FF UK, konkrétně o nástroj [Morphodita](http://ufal.mff.cuni.cz/morphodita).

S našimi daty budeme postupovat tak, že z našeho jupyter notebooku vzneseme dotaz na webovou aplikaci MorphoDity,  a požádáme na dálku o lemmatizaci a postaging našich dat (tutoriál k webovému dotazování MorpohoDity se nachází [zde](http://lindat.mff.cuni.cz/services/morphodita/api-reference.php)).


In [ ]:
sentences = tokenize.sent_tokenize(text_dokumentu)
sentences[:10]

In [ ]:
morfologicky_zpracovana_data = []
# nejprve opět vzneseme webový dotaz s textem našeho článku
for sentence in sentences:
  url = 'http://lindat.mff.cuni.cz/services/morphodita/api/tag?data=' + quote(sentence)
  resp = urlopen(url)
  resp_content = resp.read().decode("utf-8")
  # výsledky dotazu (zpracovaná data) si uložíme do proměnné "result"
  result = dict(json.loads(resp_content))["result"]
  # result má stromovou strukturu, kterou opět rozšifrujeme pomocí BeautifulSoup
  soup = BeautifulSoup(result, 'html.parser')
  # pro každé jednotlivé slovo si do jedné řádky vedle sebe vytahneme jeho (a) tvar jak je v textu, (b) lemmatizovanou podobu a (c) morfologické určení
  # tato data budeme postupně přidávat do samostatné proměnné "morfologicky_zpracovana_data"
  for token in soup.find_all("token"):
    morfologicky_zpracovana_data.append([token.get_text(), token["lemma"].partition("-")[0].partition("_")[0], token["tag"]])
# z těchto dat si vytvoříme úhlednou tabulku ve formátu DataFrame (df):
data_df = pd.DataFrame(morfologicky_zpracovana_data)
# sloupce si pojmenujeme
data_df.columns = ["slovo_puvodni", "lemma", "POS-tag"]
# výsledný dataset si vypíšeme
data_df



díky prvním písmenům třetího sloupce nyní víme, o jaký slovní druh se jedná:
*   "V..." = sloveso
*   "N..." = podstatné jméno
*   "A..." = přídavné jméno

Pro naše potřeby může bát zajímavé filtrovat tabulku tak, aby nám z textu zůstaly pouze tyto slovní druhy.


In [ ]:
data_df_filtrovana = data_df[data_df["POS-tag"].str.startswith(("V","N","A"))]
data_df_filtrovana

V této podobě může být již vcelku zajímavé podívat se na frekvence výskytů slov, resp. jejich lematizovaných tvarů. K tomu použijeme funkce z modulu "nltk".

In [ ]:
# nejprve si z naší filtrované tabulky vyxtrahujeme lemmata samotná
lemmata = data_df_filtrovana["lemma"].tolist()
# pro každý jednotlivý výraz necháme spočítat jeho počet výskytů
lemmata_freq = nltk.FreqDist(lemmata)
# vybereme např. 10 nejfrekventovanějších slov (rozumějme lemmatizovaných substantiv, adjektiv a sloves)
lemmata_most_freq = lemmata_freq.most_common(10)
print(lemmata_most_freq)

In [ ]:
# kvůli horizontálnímu zobrazení prohodíme pořadí na našem listu
lemmata_mostfreq = lemmata_most_freq
lemmata_mostfreq.reverse()

# pro potřeby grafu přiřadíme hodnoty jednotlivým osám
height = [tup[1] for tup in lemmata_mostfreq]
bars = [tup[0] for tup in lemmata_mostfreq]
y_pos = np.arange(len(bars))

plt.barh(y_pos, height)
# graf si pojmenujeme a osu také
plt.yticks(y_pos, bars)
plt.xlabel('Frekvence výskytů')
plt.title('Frekvence výskytů nejčastějších slov')
# graf si zobrazíme
plt.show()

Pokud jsme se bez chybové hlášky dostali až sem a pomocí buňky výše jsme úspěšně vykreslili graf nejfrekventovanějších termínů v námi zvoleném dokumentu z AJP, hlavní část našeho úkolu byla splněna.

Abychom doložili svůj úspěch, náš výsledný jupyter notebook si vytiskneme do PDF (nabídka File -> Print) a nahrajeme na příslušné místo do Google Classroomu.